In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import os
#import json
import torch

import time

from tqdm import tqdm

In [3]:
###setting directory
data_dir="data"
model_dir="models"
output_dir="output"
###

# Prepare data for annotation

In [ ]:
import yaml
with open('data\corona_motivations.yaml', 'r') as file:
    PVE_covid_yaml = yaml.safe_load(file)

In [ ]:
PVE_covid_yaml

In [ ]:
PVE_covid_yaml.keys()

In [ ]:
PVE_covid=pd.DataFrame()
PVE_covid['opinion']=PVE_covid_yaml['english']
PVE_covid['project']=''
PVE_covid['project'][pd.Series(PVE_covid_yaml['project'])==1]='Nursing homes allow visitors again'
PVE_covid['project'][pd.Series(PVE_covid_yaml['project'])==2]='Reopen companies (horeca and contact professions are still closed)'
PVE_covid['project'][pd.Series(PVE_covid_yaml['project'])==3]='Workers in contact professions can work again'
PVE_covid['project'][pd.Series(PVE_covid_yaml['project'])==4]='Young people do not need to maintain 1.5 meter distance among each others'
PVE_covid['project'][pd.Series(PVE_covid_yaml['project'])==5]='All restrictions are lifted for persons who are immune'
PVE_covid['project'][pd.Series(PVE_covid_yaml['project'])==6]='Restrictions are lifted in Friesland, Groningen and Drenthe'
PVE_covid['project'][pd.Series(PVE_covid_yaml['project'])==7]='Direct family members do not need to maintain 1.5 meter distance'
PVE_covid['project'][pd.Series(PVE_covid_yaml['project'])==8]='Horeca and entertainment re-open'


In [ ]:
PVE_covid_ordered=PVE_covid.sample(frac = 1,random_state=42)
PVE_covid_ordered['random_order']=list(range(PVE_covid.shape[0]))

In [ ]:
#PVE_covid_ordered
#PVE_covid_ordered.sort_index()

In [ ]:
PVE_covid_ordered['BL_batch']=np.nan
PVE_covid_ordered['AL_batch']=np.nan

PVE_covid_ordered['BL_batch'][(PVE_covid_ordered['random_order'] >=59261) & (PVE_covid_ordered['random_order']<=59460)]=-1
PVE_covid_ordered['AL_batch'][(PVE_covid_ordered['random_order'] >=59261) & (PVE_covid_ordered['random_order']<=59460)]=-1

PVE_covid_ordered['BL_batch'][PVE_covid_ordered['random_order']<100]=0
PVE_covid_ordered['AL_batch'][PVE_covid_ordered['random_order']<100]=0

PVE_covid_ordered['Autonomy_for_companies']=0
PVE_covid_ordered['Autonomy_for_close_contact_professions']=0
PVE_covid_ordered['Quality_of_life_for_elderly']=0
PVE_covid_ordered['Family_value']=0
PVE_covid_ordered['Feasibility']=0
PVE_covid_ordered['Acceptance_of_exception']=0
PVE_covid_ordered['The_need_of_schooling']=0
PVE_covid_ordered['Working_safety']=0
PVE_covid_ordered['Well-being']=0
PVE_covid_ordered['Reduce_adverse_effect']=0

In [ ]:
#PVE_covid_ordered.to_csv('data\PVE_covid_annotation_v0.csv')
PVE_covid_ordered.to_excel('data\PVE_covid_annotation_v0.xlsx')


# Finetune model: load dataset

In [4]:
#change
from datasets import Dataset
#PVE_input = Dataset.from_pandas(pd.read_csv('data\PVE_covid_annotation_v0_1.csv', encoding='latin1',index_col=0))
PVE_input =pd.read_excel('data\PVE_covid_annotation_v0_annotated.xlsx', index_col=0)
train_set=Dataset.from_pandas(PVE_input[PVE_input['BL_batch']==0])
test_set=Dataset.from_pandas(PVE_input[PVE_input['BL_batch']==-1])

In [5]:
labels = ['Autonomy_for_companies',
 'Autonomy_for_close_contact_professions',
 'Quality_of_life_for_elderly',
 'Family_value',
 'Feasibility',
 'Acceptance_of_exception',
 'The_need_of_schooling',
 'Working_safety',
 'Well-being',
 'Reduce_adverse_effect']

In [6]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [7]:
label2id 

{'Autonomy_for_companies': 0,
 'Autonomy_for_close_contact_professions': 1,
 'Quality_of_life_for_elderly': 2,
 'Family_value': 3,
 'Feasibility': 4,
 'Acceptance_of_exception': 5,
 'The_need_of_schooling': 6,
 'Working_safety': 7,
 'Well-being': 8,
 'Reduce_adverse_effect': 9}

# Preprocess data

In [8]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
    # take a batch of texts
    #text = examples["Tweet"]
    # encode them
    encoding = tokenizer(examples['project'], examples['opinion'], truncation=True)
    # add labels
    labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
    # create numpy array of shape (batch_size, num_labels)
    labels_matrix = np.zeros((len(examples['opinion']), len(labels)))
    # fill numpy array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    encoding["labels"] = labels_matrix.tolist()
  
    return encoding


In [9]:
encoded_train_set = train_set.map(preprocess_data, batched=True, remove_columns=train_set.column_names)
encoded_test_set = test_set.map(preprocess_data, batched=True, remove_columns=test_set.column_names)

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
example =encoded_train_set[0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [11]:
tokenizer.decode(example['input_ids'])

'[CLS] nursing homes allow visitors again [SEP] loneliness in the last phase of life is inhumane and also leads to premature death. [SEP]'

In [12]:
example['labels']

[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

In [13]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['Quality_of_life_for_elderly', 'Well-being']

In [14]:
if torch.cuda.is_available():
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')
    encoded_train_set.set_format("torch", device='cuda:0')
    encoded_test_set.set_format("torch", device='cuda:0')
else:
    encoded_train_set.set_format("torch")
    encoded_test_set.set_format("torch")
    print('No GPU available, using the CPU instead.')


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce MX550


# Define model

In [15]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./models/bert_train_level1/' , 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id, ignore_mismatched_sizes=True)
                                    

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./models/bert_train_level1/ and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([54, 768]) in the checkpoint and torch.Size([10, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([54]) in the checkpoint and torch.Size([10]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
if torch.cuda.is_available():
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')
    model.to('cuda')
else:    
    print('No GPU available, using the CPU instead.')


There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce MX550


# Train the model

In [17]:
batch_size = 8
metric_name = "f1"

In [18]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    #evaluation_strategy = "epoch",
    #save_strategy = "epoch",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    #learning_rate=2e-5,
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    #num_train_epochs=40,
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_steps=10,
    #push_to_hub=True,
)

In [19]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [20]:
encoded_train_set[0]['labels'].type()

'torch.cuda.FloatTensor'

In [21]:
encoded_train_set['input_ids'][0]

tensor([  101,  8329,  5014,  3499,  5731,  2153,   102, 20334,  1999,  1996,
         2197,  4403,  1997,  2166,  2003, 29582,  2063,  1998,  2036,  5260,
         2000, 21371,  2331,  1012,   102], device='cuda:0')

In [22]:
encoded_train_set['input_ids'][0].device

device(type='cuda', index=0)

In [23]:
#forward pass
outputs = model(input_ids=encoded_train_set['input_ids'][0].unsqueeze(0), labels=encoded_train_set['labels'][0].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6799, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.0886, -0.2854, -0.4828, -0.1283, -0.2782,  0.4447,  0.4155, -0.3923,
          0.6989, -0.2876]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_set,
    eval_dataset=encoded_test_set,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

***** Running training *****
  Num examples = 51
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 140
  Number of trainable parameters = 109489930
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
10,0.479900,0.360166,0.187683,0.551279,0.125000
20,0.348100,0.319162,0.294574,0.586068,0.105000
30,0.262000,0.283948,0.530021,0.695087,0.205000
40,0.211800,0.256918,0.605578,0.735906,0.290000
50,0.167200,0.288592,0.585965,0.744862,0.235000
60,0.146500,0.308044,0.584565,0.754635,0.220000
70,0.114500,0.263099,0.681600,0.817617,0.310000
80,0.098700,0.278183,0.565465,0.722728,0.230000
90,0.086100,0.265628,0.673333,0.803714,0.300000
100,0.066800,0.264949,0.601423,0.751090,0.230000


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


Training completed. Do not forge

TrainOutput(global_step=140, training_loss=0.1587786078453064, metrics={'train_runtime': 845.6928, 'train_samples_per_second': 1.206, 'train_steps_per_second': 0.166, 'total_flos': 54723321609852.0, 'train_loss': 0.1587786078453064, 'epoch': 20.0})

# Evaluate

In [26]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


{'eval_loss': 0.26699110865592957,
 'eval_f1': 0.6308492201039861,
 'eval_roc_auc': 0.7718119831713334,
 'eval_accuracy': 0.27,
 'eval_runtime': 19.7903,
 'eval_samples_per_second': 10.106,
 'eval_steps_per_second': 1.263,
 'epoch': 20.0}

# Inference

In [27]:
test_pred=trainer.predict(encoded_test_set).predictions
test_pred = torch.from_numpy(test_pred)

***** Running Prediction *****
  Num examples = 200
  Batch size = 8


In [45]:
pd.DataFrame(1*(test_pred.sigmoid()>0.5)).sum(axis=0)

0    68
1    19
2    34
3    20
4     0
5    22
6     0
7     0
8    97
9    12
dtype: int64

In [43]:
from sklearn.metrics import  classification_report

print(classification_report(PVE_input[PVE_input['BL_batch']==-1].iloc[:, 5:],np.matrix(1*(test_pred.sigmoid()>0.5))))

              precision    recall  f1-score   support

           0       0.78      0.83      0.80        64
           1       0.89      0.41      0.57        41
           2       0.59      0.95      0.73        21
           3       0.30      0.43      0.35        14
           4       0.00      0.00      0.00        14
           5       0.77      0.53      0.63        32
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00        16
           8       0.61      0.88      0.72        67
           9       0.83      0.30      0.44        33

   micro avg       0.67      0.60      0.63       305
   macro avg       0.48      0.43      0.42       305
weighted avg       0.64      0.60      0.58       305
 samples avg       0.60      0.55      0.55       305



In [44]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(PVE_input[PVE_input['BL_batch']==-1].iloc[:, 5:],np.matrix(1*(test_pred.sigmoid()>0.5)))

array([[[121,  15],
        [ 11,  53]],

       [[157,   2],
        [ 24,  17]],

       [[165,  14],
        [  1,  20]],

       [[172,  14],
        [  8,   6]],

       [[186,   0],
        [ 14,   0]],

       [[163,   5],
        [ 15,  17]],

       [[197,   0],
        [  3,   0]],

       [[184,   0],
        [ 16,   0]],

       [[ 95,  38],
        [  8,  59]],

       [[165,   2],
        [ 23,  10]]], dtype=int64)

In [30]:
model.save_pretrained('models\step0')

Configuration saved in models\step0\config.json
Model weights saved in models\step0\pytorch_model.bin


# Select cases for active sampling

In [31]:
PVE_input_selection=PVE_input[np.isnan(PVE_input['AL_batch'])]
selection_set=Dataset.from_pandas(PVE_input_selection)
#selection_set=Dataset.from_pandas(PVE_input[np.isnan(PVE_input['AL_batch'])][0:100])

In [32]:
selection_set

Dataset({
    features: ['opinion', 'project', 'random_order', 'BL_batch', 'AL_batch', 'Autonomy_for_companies', 'Autonomy_for_close_contact_professions', 'Quality_of_life_for_elderly', 'Family_value', 'Feasibility', 'Acceptance_of_exception', 'The_need_of_schooling', 'Working_safety', 'Well-being', 'Reduce_adverse_effect', '__index_level_0__'],
    num_rows: 59161
})

In [33]:
encoded_selection_set = selection_set.map(preprocess_data, batched=True, remove_columns=selection_set.column_names)

Map:   0%|          | 0/59161 [00:00<?, ? examples/s]

In [34]:
if torch.cuda.is_available():
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')
    encoded_selection_set.set_format("torch", device='cuda:0')
else:
    encoded_selection_set.set_format("torch")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce MX550


In [35]:
encoded_selection_set=encoded_selection_set.remove_columns(['labels'])

In [36]:
selection_pred=trainer.predict(encoded_selection_set).predictions
selection_pred = torch.from_numpy(selection_pred)

***** Running Prediction *****
  Num examples = 59161
  Batch size = 8


In [ ]:
#pd.DataFrame(1*(selection_pred.sigmoid()>0.5)).sum(axis=0)

In [ ]:
#pd.DataFrame(abs(selection_pred.sigmoid()-0.5)).min(axis=1)

In [ ]:
#pd.DataFrame(abs(selection_pred.sigmoid()-0.5)).min(axis=1).rank()

In [38]:
PVE_input_selection['margin']=list(pd.DataFrame(abs(selection_pred.sigmoid()-0.5)).min(axis=1))
PVE_input_selection['margin_rank']=list(pd.DataFrame(abs(selection_pred.sigmoid()-0.5)).min(axis=1).rank())

In [39]:
PVE_input_selection.to_excel('selected_case.xlsx')

In [40]:
prob=pd.DataFrame(selection_pred.sigmoid())

In [41]:
prob.to_excel('selected_case_prob.xlsx')